Import required libraries for the analysis

In [4]:
#downloaded packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#python packages
import re
import string
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

Import the train dataset and test dataset from the csv file

In [5]:
train = pd.read_csv('test.csv')
test = pd.read_csv('train.csv')

#print(len(train),len(test))

train_original=train.copy()
test_original=test.copy()

Combine Train and Test dataset for pre-processing stage

In [8]:
combine = train.append(test,ignore_index=True,sort=True)
combine.head()
combine.tail()

C:\Users\theez\AppData\Local\Temp\ipykernel_8276\2879136750.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine = train.append(test,ignore_index=True,sort=True)


,id,label,tweet
49154,31958,0.0,ate @user isz that youuu?ðððððð...
49155,31959,0.0,to see nina turner on the airwaves trying to...
49156,31960,0.0,listening to sad songs on a monday morning otw...
49157,31961,1.0,"@user #sikh #temple vandalised in in #calgary,..."
49158,31962,0.0,thank you @user for you follow
